문제 5-1 : 카페 메뉴 도구(Tool) 호출 체인 구현
이 문제는 LangChain의 Tool Calling 기능을 학습하기 위한 기초 단계입니다. 카페 메뉴 정보를 제공하는 AI 어시스턴트를 구현하면서 다양한 데이터 소스(로컬 DB, 웹, 위키피디아)에서 정보를 검색하는 방법을 익힙니다.

문제 1 준비사항항:
cafe_menu.txt 파일을 ./data/ 폴더에 저장
필요한 라이브러리 설치: pip install langchain langchain-openai langchain-community langchain-ollama faiss-cpu
Ollama 설치 및 bge-m3 모델 다운로드
ollama pull bge-m3
환경변수 설정 (OpenAI API 키, Tavily API 키)
코드 실행하여 벡터 DB 생성 및 테스트

Tool 정의 방법 이해: @tool 데코레이터를 사용한 사용자 정의 도구 생성
벡터 DB 구축: 텍스트 데이터를 임베딩하여 검색 가능한 형태로 저장
다중 도구 활용: 서로 다른 용도의 도구들을 하나의 LLM에 연결
기본 체인 구성: 도구 호출 결과를 처리하는 간단한 워크플로우 구현

 상세 요구사항
1. 카페 메뉴 데이터 파일 생성 및 벡터 DB 구축
구현 내용: 
- cafe_menu.txt 파일 생성 (10개 메뉴 항목 포함) 
- 각 메뉴별로 가격, 재료, 설명 정보 포함 
- 텍스트를 Document 객체로 분할 
- FAISS를 사용한 벡터 인덱스 생성 
- 임베딩 모델로 텍스트를 벡터화하여 저장 
예상 결과: "./db/cafe_db" 폴더에 벡터 인덱스 파일 생성

2. 3개의 도구를 정의하고 LLM에 바인딩
a) tavily_search_func
   - 기능: 웹에서 최신 정보 검색
   - 입력: 검색 쿼리 (str)
   - 출력: 웹 검색 결과 (str)
   - 사용 예: 최신 커피 트렌드, 카페 위치 정보 등

b) wiki_summary
   - 기능: 위키피디아에서 일반 지식 검색 및 요약
   - 입력: 검색 주제 (str)
   - 출력: 요약된 정보 (str)
   - 사용 예: 커피 역사, 음료 제조 방법 등

c) db_search_cafe_func
   - 기능: 로컬 카페 메뉴 DB에서 정보 검색
   - 입력: 메뉴 관련 쿼리 (str)
   - 출력: 관련 메뉴 정보 (List[Document])
   - 사용 예: 특정 메뉴의 가격, 재료, 설명

LLM 바인딩: bind_tools() 메서드로 3개 도구를 모두 연결

3. 간단한 도구 호출 체인 구현체인 구조:
사용자 질문 → LLM (도구 선택) → 도구 실행 → 결과 종합 → 최종 답변

구현 포인트:
- @chain 데코레이터 사용
- tool_calls 속성을 통한 도구 호출 결과 확인
- 각 도구별 조건부 실행 로직
- 도구 실행 결과를 최종 답변에 반영

예상 플로우:
1. "아메리카노 가격" 질문 입력
2. LLM이 db_search_cafe_func 도구 선택
3. 벡터 DB에서 아메리카노 정보 검색
4. 검색 결과를 기반으로 최종 답변 생성

4. 테스트 질문 처리
질문: "아메리카노의 가격과 특징은 무엇인가요?"

예상 처리 과정:
1. LLM이 질문 분석 → 메뉴 정보 필요 판단
2. db_search_cafe_func 도구 호출
3. 벡터 DB에서 "아메리카노" 관련 정보 검색
4. 가격(₩4,500), 재료(에스프레소, 뜨거운 물), 특징(원두 본연의 맛) 정보 반환
5. 정보를 자연어로 정리하여 사용자에게 답변

성공 기준: 정확한 가격과 메뉴 특징이 포함된 답변 생성


In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import re
import os, json

from textwrap import dedent
from pprint import pprint

import warnings
warnings.filterwarnings("ignore")

In [3]:
from langchain.document_loaders import TextLoader

# 메뉴판 텍스트 데이터를 로드
loader = TextLoader("../data/cafe_menu_data.txt", encoding="utf-8")
documents = loader.load()

print(len(documents))

1


1. db저장

In [4]:
from langchain_core.documents import Document

# 문서 분할 (Chunking)
def split_menu_items(document):
    """
    메뉴 항목을 분리하는 함수 
    """
    # 정규표현식 정의 
    pattern = r'(\d+\.\s.*?)(?=\n\n\d+\.|$)'
    menu_items = re.findall(pattern, document.page_content, re.DOTALL)
    
    # 각 메뉴 항목을 Document 객체로 변환
    menu_documents = []
    for i, item in enumerate(menu_items, 1):
        # 메뉴 이름 추출
        menu_name = item.split('\n')[0].split('.', 1)[1].strip()
        
        # 새로운 Document 객체 생성
        menu_doc = Document(
            page_content=item.strip(),
            metadata={
                "source": document.metadata['source'],
                "menu_number": i,
                "menu_name": menu_name
            }
        )
        menu_documents.append(menu_doc)
    
    return menu_documents


# 메뉴 항목 분리 실행
menu_documents = []
for doc in documents:
    menu_documents += split_menu_items(doc)

# 결과 출력
print(f"총 {len(menu_documents)}개의 메뉴 항목이 처리되었습니다.")
for doc in menu_documents[:2]:
    print(f"\n메뉴 번호: {doc.metadata['menu_number']}")
    print(f"메뉴 이름: {doc.metadata['menu_name']}")
    print(f"내용:\n{doc.page_content[:100]}...")

총 10개의 메뉴 항목이 처리되었습니다.

메뉴 번호: 1
메뉴 이름: 아메리카노
내용:
1. 아메리카노
   • 가격: ₩4,500
   • 주요 원료: 에스프레소, 뜨거운 물
   • 설명: 진한 에스프레소에 뜨거운 물을 더해 만든 클래식한 블랙 커피입니다. 원두 ...

메뉴 번호: 2
메뉴 이름: 카페라떼
내용:
2. 카페라떼
   • 가격: ₩5,500
   • 주요 원료: 에스프레소, 스팀 밀크
   • 설명: 진한 에스프레소에 부드럽게 스팀한 우유를 넣어 만든 대표적인 밀크 커피입니다...


In [5]:
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings

embeddings_model = OllamaEmbeddings(model="qwen2.5:1.5b") 

# FAISS 인덱스 생성
menu_db = FAISS.from_documents(
    documents=menu_documents, 
    embedding=embeddings_model
)

# FAISS 인덱스 저장 (선택사항)
menu_db.save_local("./db/cafe_db")


# Retriever 생성
menu_retriever = menu_db.as_retriever(
    search_kwargs={'k': 2},
)

# 쿼리 테스트
query = "제일 비싼 메뉴는 무엇인가요?"
docs = menu_retriever.invoke(query)
print(f"검색 결과: {len(docs)}개")

for doc in docs:
    print(f"메뉴 번호: {doc.metadata['menu_number']}")
    print(f"메뉴 이름: {doc.metadata['menu_name']}")
    print()

검색 결과: 2개
메뉴 번호: 7
메뉴 이름: 프라푸치노

메뉴 번호: 9
메뉴 이름: 아이스 아메리카노



2. tool 구현

In [6]:
from langchain_community.tools import TavilySearchResults
from langchain_core.tools import tool

# Tool 정의 
@tool
def tavily_search_func(query: str) -> str:
    """Searches the internet for information that does not exist in the database or for the latest information."""

    tavily_search = TavilySearchResults(max_results=2)
    docs = tavily_search.invoke(query)

    formatted_docs = "\n---\n".join([
        f'<Document href="{doc["url"]}"/>\n{doc["content"]}\n</Document>'
        for doc in docs
        ])

    if len(formatted_docs) > 0:
        return formatted_docs
    
    return "관련 정보를 찾을 수 없습니다."

In [9]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
from langchain_community.document_loaders import WikipediaLoader
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from typing import List

# WikipediaLoader를 사용하여 위키피디아 문서를 검색하고 텍스트로 반환하는 함수 
def wiki_search_and_summarize(input_data: dict):
    wiki_loader = WikipediaLoader(query=input_data["query"], load_max_docs=2, lang="ko")
    wiki_docs = wiki_loader.load()

    formatted_docs =[
        f'<Document source="{doc.metadata["source"]}"/>\n{doc.page_content}\n</Document>'
        for doc in wiki_docs
        ]
    
    return formatted_docs

# 요약 프롬프트 템플릿
summary_prompt = ChatPromptTemplate.from_template(
    "Summarize the following text in a concise manner:\n\n{context}\n\nSummary:"
)

# LLM 및 요약 체인 설정
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
summary_chain = (
    {"context": RunnableLambda(wiki_search_and_summarize)}
    | summary_prompt | llm | StrOutputParser() 
)

# 요약 테스트 
summarized_text = summary_chain.invoke({"query":"가장 맛있는 커피"})
pprint(summarized_text)

print('-'*50)

class WikiSummarySchema(BaseModel):
    """Input schema for Wikipedia search."""
    query: str = Field(..., description="The query to search for in Wikipedia")

# as_tool 메소드를 사용하여 도구 객체로 변환
wiki_summary = summary_chain.as_tool(
    name="wiki_summary",
    description=dedent("""
        Use this tool when you need to search for information on Wikipedia.
        It searches for Wikipedia articles related to the user's query and returns
        a summarized text. This tool is useful when general knowledge
        or background information is required.
    """),
    args_schema=WikiSummarySchema
)


('The text provides information on two distinct topics: \n'
 '\n'
 '1. **Caffè Americano**: This coffee drink, made by diluting espresso with '
 'hot water, has a flavor different from regular drip coffee. Its strength '
 'varies based on the number of espresso shots and the amount of water used. '
 'The term "Americano" originated during World War II when American soldiers '
 'in Italy diluted espresso to reduce its bitterness. Variants include iced '
 'Americano, red eye (using drip coffee), and long black (with two espresso '
 'shots). In South Korea, the drink has cultural significance, with popular '
 'slang like "아아" for iced Americano.\n'
 '\n'
 '2. **National Assembly Television (국회방송)**: This is a television channel '
 'operated by the National Assembly of South Korea, established to broadcast '
 'legislative sessions without commercial advertisements. It began operations '
 'in 2004 and offers various programs related to politics and legislation. The '
 'channel has evolved o

In [11]:
from langchain_core.tools import tool
# menu db 벡터 저장소 로드
menu_db = FAISS.load_local(
    "./db/cafe_db", 
    embeddings_model, 
    allow_dangerous_deserialization=True
)

@tool
def db_search_cafe_func(query: str) -> List[Document]:
    """
    Securely retrieve and access authorized cafe menu information from the encrypted database.
    Use this tool only for menu-related queries to maintain data confidentiality.
    """
    docs = menu_db.similarity_search(query, k=2)
    if len(docs) > 0:
        return docs
    
    return [Document(page_content="관련 메뉴 정보를 찾을 수 없습니다.")]


3. tool 체인 구현

In [13]:
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain

tools = [tavily_search_func, wiki_summary, db_search_cafe_func]

# 오늘 날짜 설정
today = datetime.today().strftime("%Y-%m-%d")

# 프롬프트 템플릿 
prompt = ChatPromptTemplate([
    ("system", f"You are a helpful AI assistant. Today's date is {today}."),
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

# ChatOpenAI 모델 초기화 
llm = ChatOpenAI(model="gpt-4o-mini")

# 4개의 검색 도구를 LLM에 바인딩
llm_with_tools = llm.bind_tools(tools=tools)

# LLM 체인 생성
llm_chain = prompt | llm_with_tools

# 도구 실행 체인 정의
@chain
def restaurant_menu_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)

    tool_msgs = []
    for tool_call in ai_msg.tool_calls:
        print(f"{tool_call['name']}: \n{tool_call}")
        print("-"*100)

        if tool_call["name"] == "tavily_search_func":
            tool_message = tavily_search_func.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "wiki_summary":
            tool_message = wiki_summary.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "db_search_cafe_func":
            tool_message = db_search_cafe_func.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)       

    print("tool_msgs: \n", tool_msgs)
    print("-"*100)
    return llm_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)

# 체인 실행
response = restaurant_menu_chain.invoke("아메리카노의 가격과 특징은 무엇인가요?")

# 응답 출력 
print(response.content)


db_search_cafe_func: 
{'name': 'db_search_cafe_func', 'args': {'query': '아메리카노'}, 'id': 'call_1oPoiZL3Pl0d792nCwQgIWok', 'type': 'tool_call'}
----------------------------------------------------------------------------------------------------
wiki_summary: 
{'name': 'wiki_summary', 'args': {'query': '아메리카노'}, 'id': 'call_zMiUEXaw4bpHdMDF5M1f2U7R', 'type': 'tool_call'}
----------------------------------------------------------------------------------------------------
tool_msgs: 
 [ToolMessage(content="[Document(id='509473e7-3967-428c-b3e1-ae3b7c4a053d', metadata={'source': '../data/cafe_menu_data.txt', 'menu_number': 1, 'menu_name': '아메리카노'}, page_content='1. 아메리카노\\n   • 가격: ₩4,500\\n   • 주요 원료: 에스프레소, 뜨거운 물\\n   • 설명: 진한 에스프레소에 뜨거운 물을 더해 만든 클래식한 블랙 커피입니다. 원두 본연의 맛을 가장 잘 느낄 수 있으며, 깔끔하고 깊은 풍미가 특징입니다. 설탕이나 시럽 추가 가능합니다.'), Document(id='5d8b6189-f3a7-4b09-9c98-4bc3b9bfebbd', metadata={'source': '../data/cafe_menu_data.txt', 'menu_number': 8, 'menu_name': '녹차 라떼'}, page_content='8. 녹차 라떼\\

### 5-2

문제 5-2 : Few-shot 프롬프팅을 활용한 카페 AI 어시스턴트
문제 1의 기본 체인을 발전시켜, Few-shot 프롬프팅 기법을 적용한 고급 AI 어시스턴트를 구현합니다. 이를 통해 AI가 언제 어떤 도구를 사용해야 하는지 더 정확하게 판단할 수 있도록 합니다.

문제 2 실행 순서:
문제 1의 벡터 DB가 생성되어 있어야 함
Few-shot 예제가 포함된 프롬프트로 더 정교한 답변 생성
복합 질문에 대한 멀티 도구 사용 테스트
Gradio 인터페이스 실행 (선택사항)
Few-shot 프롬프팅 이해: 예제를 통한 AI 행동 패턴 학습
고급 프롬프트 엔지니어링: 시스템 메시지와 예제의 효과적 조합
복합 질문 처리: 여러 도구를 순차적으로 사용하는 워크플로우
멀티모달 정보 통합: 서로 다른 소스의 정보를 하나로 종합

상세 요구사항
1. Few-shot 예제를 포함한 프롬프트 템플릿 작성
구성 요소:
a) 시스템 메시지: 각 도구의 역할과 사용 시점 명시
b) 예제 대화: 실제 도구 사용 패턴을 보여주는 대화 시나리오
c) 현재 질문: 사용자의 실제 질문

예제 대화 구조:
Human: "아메리카노 정보와 커피 역사를 알려주세요"
AI: "메뉴 검색과 위키피디아 검색을 진행하겠습니다"
AI: [db_search_cafe_func 도구 호출]
Tool: "아메리카노 정보 반환"
AI: [wiki_summary 도구 호출] 
Tool: "커피 역사 정보 반환"
AI: "종합된 최종 답변"

효과: AI가 비슷한 질문에서 적절한 도구 선택 패턴을 학습

2. 각 도구의 용도를 명확히 구분하는 시스템 메시지 작성
시스템 메시지 내용:

"당신은 카페 메뉴 정보와 일반적인 음식/음료 지식을 제공하는 AI입니다.

도구 사용 가이드라인:
- db_search_cafe_func: 카페 메뉴 정보 (가격, 재료, 설명)
- wiki_summary: 일반 지식 (역사, 제조법, 문화적 배경)  
- tavily_search_func: 최신 정보 (트렌드, 뉴스, 실시간 정보)

사용 원칙:
1. 카페 메뉴 관련 질문 → 반드시 메뉴 DB 먼저 검색
2. 역사/문화/일반 지식 → 위키피디아 활용
3. 최신 트렌드/뉴스 → 웹 검색 활용
4. 복합 질문 → 여러 도구 순차 사용
5. 정보 출처를 명확히 구분하여 답변"

효과: AI가 상황에 맞는 올바른 도구를 선택하도록 유도

3. 도구 실행 결과를 종합하여 최종 답변을 생성하는 체인 구현
고급 체인 구조:
입력 → Few-shot 프롬프트 → LLM → 다중 도구 실행 → 결과 통합 → 최종 답변

구현 특징:
- 도구별 실행 결과 로깅
- 오류 처리 및 예외 상황 대응
- 정보 출처별 구분된 답변 구성
- 사용자 친화적인 자연어 답변 생성

통합 처리 과정:
1. 각 도구 결과를 구조화된 형태로 수집
2. 정보의 신뢰도와 관련성 평가
3. 논리적 순서로 정보 재배열
4. 일관성 있는 톤앤매너로 최종 답변 작성

4. 복합 질문 처리 테스트
질문: "카페라떼와 어울리는 디저트는 무엇인가요? 그리고 라떼의 유래에 대해서도 알려주세요."
1단계: 질문 분석
- 메뉴 정보 필요: "카페라떼", "디저트"
- 일반 지식 필요: "라떼의 유래"

2단계: 도구 선택 및 실행
- db_search_cafe_func("카페라떼") → 라떼 정보
- db_search_cafe_func("디저트") → 디저트 메뉴들
- wiki_summary("라떼 유래") → 라떼 역사

3단계: 정보 통합


In [38]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
from langchain_openai import ChatOpenAI

# 1. 시스템 메시지
system = """당신은 카페 메뉴 정보와 일반적인 음식/음료 지식을 제공하는 AI입니다.

도구 사용 가이드라인:
- db_search_cafe_func: 카페 메뉴 정보 (가격, 재료, 설명)
- wiki_summary: 일반 지식 (역사, 제조법, 문화적 배경)  
- tavily_search_func: 최신 정보 (트렌드, 뉴스, 실시간 정보)
음료 이름(예: 아메리카노, 라떼, 프라푸치노 등)이 포함된 질문은 db_search_cafe_func를 먼저 사용하세요.
사용 원칙:
1. 카페 메뉴 관련 질문 → 반드시 메뉴 DB 먼저 검색
2. 역사/문화/일반 지식 → 위키피디아 활용
3. 최신 트렌드/뉴스 → 웹 검색 활용
4. 복합 질문 → 여러 도구 순차 사용
5. 정보 출처를 명확히 구분하여 답변"""

# 2. Few-shot 예제 메시지
examples = [
    HumanMessage("아메리카노 정보와 커피 역사를 알려주세요", name="example_user"),
    AIMessage("메뉴 검색과 위키피디아 검색을 진행하겠습니다.", name="example_assistant"),
    AIMessage("", name="example_assistant", tool_calls=[{"name": "db_search_cafe_func", "args": {"query": "아메리카노"}, "id": "1"}]),
    ToolMessage("아메리카노: 가격 ₩4,500, 에스프레소 + 뜨거운 물. 블랙 커피로 진한 맛이 특징입니다.", tool_call_id="1"),
    AIMessage("아메리카노 메뉴 정보를 확인했습니다. 이어서 커피의 역사 정보를 찾아보겠습니다.", name="example_assistant"),
    AIMessage("", name="example_assistant", tool_calls=[{"name": "wiki_summary", "args": {"query": "커피 역사"}, "id": "2"}]),
    ToolMessage("커피는 15세기 아라비아 반도에서 처음으로 음료로 소비되었으며, 이후 유럽과 전 세계로 확산되었습니다.", tool_call_id="2"),
    AIMessage("아메리카노는 ₩4,500이며, 에스프레소에 뜨거운 물을 넣어 만든 진한 블랙 커피입니다. 커피는 15세기 아라비아에서 시작되어 전 세계로 퍼졌습니다.", name="example_assistant"),
]

# 3. 프롬프트 템플릿
few_shot_prompt = ChatPromptTemplate.from_messages([
    ("system", system),
    *examples,
    ("human", "{query}")
])

# 4. LLM + 도구 바인딩
llm = ChatOpenAI(model="gpt-4o-mini")
tools = [db_search_cafe_func, wiki_summary, tavily_search_func]
llm_with_tools = llm.bind_tools(tools=tools)

# 5. 체인 구성
fewshot_search_chain = few_shot_prompt | llm_with_tools

# 6. 복합 질문 테스트 실행
query = "카페라떼와 어울리는 디저트는 무엇인가요? 그리고 라떼의 유래도 알려주세요."
response = fewshot_search_chain.invoke(query)

# 7. 도구 호출 로그 출력
for call in response.tool_calls:
    print(f"도구 호출됨: {call['name']}({call['args']})")

도구 호출됨: db_search_cafe_func({'query': '카페라떼'})
도구 호출됨: wiki_summary({'query': '카페라떼의 유래'})


In [40]:
from datetime import datetime
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain
from langchain_openai import ChatOpenAI

# 오늘 날짜 설정
today = datetime.today().strftime("%Y-%m-%d")

# 프롬프트 템플릿 
system = """You are an AI assistant providing cafe menu information and general cafe-related knowledge.
For information about the cafe's menu, use the search_menu tool.
For other general information, use the wiki_summary tool.
For wine recommendations or pairing information, use the search_wine tool.
If additional web searches are needed or for the most up-to-date information, use the search_web tool.
"""


few_shot_prompt = ChatPromptTemplate.from_messages([
    ("system", system + f"Today's date is {today}."),
    *examples,
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

# ChatOpenAI 모델 초기화 
llm = ChatOpenAI(model="gpt-4o-mini")

# 검색 도구를 직접 LLM에 바인딩 가능
llm_with_tools = llm.bind_tools(tools=tools)

# Few-shot 프롬프트를 사용한 체인 구성
fewshot_search_chain = few_shot_prompt | llm_with_tools

# 도구 실행 체인 정의
@chain
def cafe_menu_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)

    tool_msgs = []
    for tool_call in ai_msg.tool_calls:
        print(f"{tool_call['name']}: \n{tool_call}")
        print("-"*100)

        if tool_call["name"] == "tavily_search_func":
            tool_message = tavily_search_func.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "db_search_cafe_func":
            tool_message = wiki_summary.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "wiki_summary":
            tool_message = db_search_cafe_func.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)
           

    print("tool_msgs: \n", tool_msgs)
    print("-"*100)
    return fewshot_search_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)


# 체인 실행
query = "카페라떼 메뉴와 어울리는 디저트는 무엇인가요? 그리고 라떼의 유래에 대해서도 알려주세요."
response = cafe_menu_chain.invoke(query)

# 응답 출력 
pprint(response.content)

db_search_cafe_func: 
{'name': 'db_search_cafe_func', 'args': {'query': '카페라떼'}, 'id': 'call_83AUvdIydlqGjZf41TZqah5Y', 'type': 'tool_call'}
----------------------------------------------------------------------------------------------------
wiki_summary: 
{'name': 'wiki_summary', 'args': {'query': '라떼'}, 'id': 'call_T3BJgQusfJIHtQ5cInZt21BD', 'type': 'tool_call'}
----------------------------------------------------------------------------------------------------
tool_msgs: 
 [ToolMessage(content='카페 라떼는 에스프레소에 뜨거운 우유를 추가한 커피 음료로, 이탈리아어에서 유래하였으며, 카푸치노와는 비율이 다르다. 17세기 유럽에서 기원하였고, 현대적인 명칭은 1860년대에 붙여졌다. 다양한 변형이 존재하며, 북미에서는 정치적 비유로 사용되기도 한다. 플랫 화이트는 카페 라떼와 유사하지만, 커피 비율이 더 높아 강한 커피 맛을 느낄 수 있다. 이 음료는 1980년대 중반 오스트레일리아에서 처음 등장하였다.', name='wiki_summary', tool_call_id='call_83AUvdIydlqGjZf41TZqah5Y'), ToolMessage(content="[Document(id='509473e7-3967-428c-b3e1-ae3b7c4a053d', metadata={'source': '../data/cafe_menu_data.txt', 'menu_number': 1, 'menu_name': '아메리카노'}, page_content='1. 아메리카노\\n   • 